In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [3]:
# Read in data tables
engagement = pd.read_csv('takehome_user_engagement.csv')
users = pd.read_csv('takehome_users.csv',encoding="latin-1")

In [13]:
# Check general form of tables
engagement.head(20)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1


In [8]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [11]:
print(f'{len(users)} users logged in a total of {len(engagement)} times.')

12000 users logged in a total of 207917 times.


In [22]:
# Let's try completing this one with SQL instead.
engine = create_engine('sqlite:///sqlengine.db')

In [23]:
engagement.to_sql('engagement',engine)
users.to_sql('users',engine)

12000

In [24]:
# Every login counts for only one visit
pd.read_sql('SELECT DISTINCT visited FROM engagement',engine)

,visited
0,1


In [26]:
# This shows that not every user ever logs in on the engagement table
pd.read_sql('SELECT DISTINCT user_id FROM engagement LIMIT 10', engine)

,user_id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,10
8,11
9,13


In [31]:
pd.read_sql('SELECT user_id, SUM(visited) AS visits FROM engagement GROUP BY user_id HAVING visits >= 3',engine)

,user_id,visits
0,2,14
1,10,284
2,20,7
3,33,18
4,42,342
...,...,...
2243,11975,216
2244,11980,8
2245,11981,4
2246,11988,30


In [ ]:
pd.read_sql('CREATE TABLE active AS SELECT user_id, SUM(visited) AS "visits" FROM engagement GROUP BY user_id HAVING visits >= 3',engine)

In [36]:
pd.read_sql('SELECT * FROM active LIMIT 10',engine)

,user_id,visits
0,2,14
1,10,284
2,20,7
3,33,18
4,42,342
5,43,8
6,50,17
7,53,9
8,59,3
9,60,8


In [71]:
# Total logins by only users who have logged in more than 3 times
total_active_logins = pd.read_sql('SELECT COUNT(*) FROM engagement WHERE user_id IN (SELECT user_id FROM active)',engine).values[0][0]
total_active_logins

201002

In [72]:
# Total users who have logged in more than 3 times
total_active_users = pd.read_sql('SELECT COUNT(DISTINCT(user_id)) FROM active', engine).values[0][0]
total_active_users

2248

In [73]:
# Total logins by all users
total_logins = pd.read_sql('SELECT COUNT(*) FROM engagement',engine).values[0][0]
total_logins

207917

In [75]:
# Total number of users
total_users = pd.read_sql('SELECT COUNT(DISTINCT object_id) FROM users', engine).values[0][0]
total_users

12000

In [84]:
# Number of users who never logged in
inactive_users = pd.read_sql('SELECT COUNT(*) FROM users WHERE object_id NOT IN (SELECT user_id FROM engagement)',engine).values[0][0]
inactive_users

3177

In [97]:
print(f"To sum up, of the {total_logins} total logins by {total_users} total users, {total_active_logins} \
({round(total_active_logins/total_logins*100)}% of logins)\nwere done by the {total_active_users} \
users who had logged in more than 3 times (just {round(total_active_users/total_users*100)}% of users). \n\
In fact, {inactive_users} users ({round(inactive_users/total_users*100)}% of users who created an account) \
never logged in at all!")

To sum up, of the 207917 total logins by 12000 total users, 201002 (97% of logins)
were done by the 2248 users who had logged in more than 3 times (just 19% of users). 
In fact, 3177 users (26% of users who created an account) never logged in at all!


In [101]:
print(f"The average number of logins per user is {round(total_logins/total_users,1)}.")
print(f"The average number of logins per user who logged in at least once is {round(total_logins/(total_users - inactive_users),1)}.")
print(f"The average number of logins per user who logged in at least 3 times is {round(total_active_logins/total_active_users,1)}.")

The average number of logins per user is 17.3.
The average number of logins per user who logged in at least once is 23.6.
The average number of logins per user who logged in at least 3 times is 89.4.
